In [2]:

from battle import *
from numpy.random import random, randint
from termcolor import colored
# (position=[0,0])
chessCanBeUsed = [ant,
                  llama,
                  wolf, 
                  ladybug, 
                  bee, 
                  swallower, 
                  sea_hedgehog, 
                  heal_deer, 
                  hippo, 
                  bear, 
                  butterfly, 
                  fireworm, 
                  mantis, 
                  scorpion, 
                  anglerfish, 
                  tiger, 
                  shark, 
                  spider, 
                  rabbit, 
                #   anglerfish(position=[0,0])
                  
                  ]
cost1Card = [ant, littleUglyFish, rabbit]
cost2Card = [llama, wolf, ladybug, bee, swallow, sea_hedgehog]
unavail = {electric_eel,monoceros,turtle,elephant,unicorn_b,littleUglyFish,monkey} # octopus
# 初始棋子
redTeam = []
blueTeam = []
class Player:
  # id = 0
  def __init__(self, id) -> None:
    # self.id = Player.id
    # Player.id += 1
    self.id = id
    self.round:int = 1
    self.chesses: dict[int, chessInterface] = {}
    self.chessInHand: list[chessInterface] = [] # max Of 5 cards
    self.chessOnField:list[chessInterface] = []
    self.goldAvail = 1
    self.buffs = []
    self.population = 1
    self.roundToNextPop = 1
    self.goldToNextPop = 1
    self.cost2Chance = 0 # 抽到2星卡的概率
    self.hp = 100

  def __repr__(self) -> str:
    if self.id == 0:
      color = 'red'
    else: 
      color = 'blue'
    return f"{colored('玩家'+str(self.id),color)}"
  
  def printStatus(self) -> None:
    print (f"\n" + \
          f"当前回合 {self.round}, 剩余血量：{self.hp}\n" + \
          f"{self} 现在拥有 {colored(self.population,'green')} 人口，距离下次升级还需要{colored(self.roundToNextPop,'light_cyan')}回合，或者{colored(self.roundToNextPop*self.roundToNextPop,'light_cyan')}金钱；\n" + \
          f"现在拥有的棋子为{self.chesses}\n" + \
          f"备战区棋子：{self.chessInHand}，上场棋子{self.chessOnField}；\n"+ \
          f"当前回合还有可用金钱{colored(self.goldAvail, 'yellow')}；\n" + \
          f"玩家buff有{self.buffs}；\n" + \
          "\n")

  def next_round(self):
    '''每回合开始阶段调用这个方法'''
    self.hp -= 10
    self.round += 1
    self.goldAvail = deepcopy(self.round)
    self.checkPopulationUpdate()      
    self.printStatus()  
  
# 金钱部分
  def reduce_gold(self, howMuch, purpose: str):
    '''花钱的时候调用这个方法'''
    if self.goldAvail - howMuch >= 0:
      self.goldAvail -= howMuch
      print(self,purpose,f"{colored('成功','green')},共花费: ",howMuch," 金币，还剩 ",self.goldAvail," 金币。")
      return True
    else:
      print(self,purpose,f"{colored('失败','red')},需要: ",howMuch,' 金币，现有 ',self.goldAvail,f" 金币，差{howMuch - self.goldAvail}金币。")
      return False
    
  def checkPopulationUpdate(self):
    """检查人口是否上升"""
    self.roundToNextPop -= 1
    self.goldToNextPop -= self.population
    if self.roundToNextPop == 0:
      self.population += 1
      self.roundToNextPop = deepcopy(self.population)
      self.costForNextPop = deepcopy(self.population)
      print(f"玩家{self.id} 现在拥有 {colored(self.population,'green')} 人口，升级至下一还需要{colored(self.roundToNextPop,'light_cyan')}回合，或者 {colored(self.roundToNextPop*self.roundToNextPop,'light_cyan')} 金钱；\n")
      
# 人口部分
  def upgrade_pop(self):
    """用金币升级人口"""
    if self.reduce_gold(purpose=f"升级人口", howMuch=self.population):
      self.checkPopulationUpdate()
      
# 棋子部分
  def printChesses(self):
    print(self.chesses)
    
  def get_random_chess_to_draw(self):
    """每回合随机抽卡"""
    cardList = []
    for i in range(3):
      if random() < self.cost2Chance:
        cardList.append( cost2Card[randint(0,len(cost2Card))] )
      else:
        cardList.append( cost1Card[randint(0,len(cost1Card))] )
    print(self,"本次选择有",cardList)
    return cardList
  def redraw(self):
    """重新抽卡"""
    if self.reduce_gold(purpose=f'刷新棋子', howMuch = 3):
      self.get_random_chess_to_draw()
    
  def checkChessID(self, chessID:int) -> chessInterface:
    try:
      chess = self.chesses[chessID]
    except:
      raise KeyError(f"{self}没有这张卡")
    return chess

  def get_chess(self,newChess: chessInterface):
    """购买棋子"""
    newChess.team = self.id
    if self.reduce_gold(purpose=f"购买棋子{newChess}", howMuch=newChess.star):
      self.chessInHand.append(newChess)
      self.chesses[newChess.uniqueID] = newChess 

  def sell_chess(self, chessID: int):
    """出售棋子"""
    chess = self.checkChessID(chessID=chessID)
    if self.chesses[chess]:
      self.chessOnField.remove(chess)
      self.goldAvail += chess.star/2
      print(f"{self}:已被卖掉，从chessOnField and chesses中移除,{self}获得金钱{chess.star/2}，现在闲置金钱{self.goldAvail}")
    else:
      self.chessInHand.remove(chess)
      self.goldAvail += chess.star/2
      print(f"{self}:已被卖掉，从chessInHand and chesses中移除,{self}获得金钱{chess.star/2}，现在闲置金钱{self.goldAvail}")
    for (s, c) in self.chesses.items():
      if c == chess:
        break
    del self.chesses[s]
    print(  f"       现在拥有的棋子为{self.chesses}")
    print(  f"       备战区棋子：{self.chessInHand}，\n  上场棋子{self.chessOnField}；")
      
  def hand_to_field(self, chessID: int, position: list[int]):
    """上场棋子"""
    
    chess = self.checkChessID(chessID=chessID)
    if chess in self.chessOnField:
      print(f"棋子{chess}已经在场上了，请勿重复上场!")
      return
    self.chessInHand.remove(chess)
    self.chessOnField.append(chess)
    chess.setInitialPosition(position = position)
    chess.onField = True
    print(f"{self}:{chess}上场到位置{colored(str(chess.position), color='magenta')}")
    print(f"       备战区棋子：{self.chessInHand}，\n       上场棋子{self.chessOnField}；")
    
  def field_to_hand(self, chessID: int):
    """下场棋子"""
    chess = self.checkChessID(chessID=chessID)
    if chess in self.chessInHand:
      print(f"棋子{chess}已经在备战区了，不能下场！")
      return
    chess.position = [-1,-1]
    self.chessOnField.remove(chess)
    self.chessInHand.append(chess)
    chess.onField = False
    print(f"{self}:{chess}下场")
    print(f"       备战区棋子：{self.chessInHand}，\n       上场棋子{self.chessOnField}；")
    
  
# 棋子升级
  def first_2_copy(self, chessID: int) -> list[chessInterface]:
    chess = self.checkChessID(chessID=chessID)
    sameChess = []
    for c in self.chesses.values():
      if c.id == chess.id:
        sameChess.append(c)
    if len(sameChess) >= 2:
      return sameChess[:2]
    else:
      return []

  def upgrade_chess(self, chessID: int ):
    chess = self.checkChessID(chessID=chessID)
    sameChess = self.first_2_copy(chess)
    if len(sameChess) >= 2:
      for c in sameChess:
        if c.onField:
          self.chessOnField.remove(c)
        else:
          self.chessInHand.remove(c)
        del self.chesses[c.uniqueID]
        chess.onField = False
      print(f"  {self}:{chess}升级{colored('成功','green')}，请选择一个 {chess.star + 1} 星棋子")
    else:
      print(f"  {self}:{chess}升级{colored('失败','red')}，请检查棋子持有数量")
        
      
  # def get_chess(self,chess:chessInterface):
  #   if chess.id in [1,2,3]:
  #     self.goldAvail -= 1
  #   elif chess.id in [4,5,6,7,8,9]:
  #     self.goldAvail -= 2

  

In [3]:
# 初始化玩家
player1 = Player(0)
player2 = Player(1)

回合1


每人1棋子

In [4]:
player1.printStatus()
player2.printStatus()
player1.get_random_chess_to_draw()
player2.get_random_chess_to_draw()




当前回合 1, 剩余血量：100
玩家0 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{}
备战区棋子：[]，上场棋子[]；
当前回合还有可用金钱1；
玩家buff有[]；



当前回合 1, 剩余血量：100
玩家1 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{}
备战区棋子：[]，上场棋子[]；
当前回合还有可用金钱1；
玩家buff有[]；


玩家0 本次选择有 [<class 'chess_info.ant'>, <class 'chess_info.rabbit'>, <class 'chess_info.littleUglyFish'>]
玩家1 本次选择有 [<class 'chess_info.rabbit'>, <class 'chess_info.ant'>, <class 'chess_info.rabbit'>]


[chess_info.rabbit, chess_info.ant, chess_info.rabbit]

In [5]:
# 选择棋子
player1.get_chess(rabbit())
player2.get_chess(ant())

player1.printChesses()
player2.printChesses()


玩家0 购买棋子兔子1 成功,共花费:  1  金币，还剩  0  金币。
玩家1 购买棋子蚂蚁2 成功,共花费:  1  金币，还剩  0  金币。
{1: 兔子1}
{2: 蚂蚁2}


In [6]:
# 准备阶段
player1Chess = 1
player2Chess = 2

# 上场棋子
player1.hand_to_field(player1Chess, position=[2,2])
player2.hand_to_field(player2Chess, position=[3,2])

# 下场棋子

# player1.field_to_hand(player1Chess)
# player2.field_to_hand(player2Chess)

# player1.printStatus()
# player2.printStatus()


玩家0:兔子1上场到位置[2, 2]
       备战区棋子：[]，
       上场棋子[兔子1]；
玩家1:蚂蚁2上场到位置[3, 2]
       备战区棋子：[]，
       上场棋子[蚂蚁2]；

当前回合 1, 剩余血量：100
玩家0 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{1: 兔子1}
备战区棋子：[]，上场棋子[兔子1]；
当前回合还有可用金钱0；
玩家buff有[]；



当前回合 1, 剩余血量：100
玩家1 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{2: 蚂蚁2}
备战区棋子：[]，上场棋子[蚂蚁2]；
当前回合还有可用金钱0；
玩家buff有[]；




In [7]:
# 战斗阶段
round1Battle = battle()
round1Battle.addRedTeam(redTeam= player1.chessOnField)
round1Battle.addBlueTeam(blueTeam= player2.chessOnField)
round1Battle.battle_with_skills()



[None, None, None, None, None]

[None, None, None, None, None]

[None, None, 兔子1, None, None]

[None, None, 蚂蚁2, None, None]

[None, None, None, None, None]

[None, None, None, None, None]


0.9 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:157.89473684210526 / 180
1.0 <蚂蚁2> 攻击了<兔子1>, 造成了22.099447513812155点伤害，<兔子1>生命值还剩:295.90055248618785 / 318
1.8 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:135.78947368421052 / 180
2.0 <蚂蚁2> 攻击了<兔子1>, 造成了22.099447513812155点伤害，<兔子1>生命值还剩:273.8011049723757 / 318
2.7 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:113.68421052631578 / 180
3.0 <蚂蚁2> 攻击了<兔子1>, 造成了22.099447513812155点伤害，<兔子1>生命值还剩:251.70165745856355 / 318
3.6 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:91.57894736842104 / 180
4.0   兔子1对蚂蚁2使用了*回手掏*,造成100点伤害
4.0   蚂蚁2被【眩晕】了！

    蚂蚁2 已经被打败!

红方获胜!

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, 兔子1, None, None]

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, Non

回合2


In [8]:
player1.next_round()
player2.next_round()

player1.get_random_chess_to_draw()
player2.get_random_chess_to_draw()

玩家0 现在拥有 2 人口，升级至下一还需要2回合，或者 4 金钱；


当前回合 2, 剩余血量：90
玩家0 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{1: 兔子1}
备战区棋子：[]，上场棋子[兔子1]；
当前回合还有可用金钱2；
玩家buff有[]；


玩家1 现在拥有 2 人口，升级至下一还需要2回合，或者 4 金钱；


当前回合 2, 剩余血量：90
玩家1 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{2: 蚂蚁2}
备战区棋子：[]，上场棋子[蚂蚁2]；
当前回合还有可用金钱2；
玩家buff有[]；


玩家0 本次选择有 [<class 'chess_info.rabbit'>, <class 'chess_info.rabbit'>, <class 'chess_info.littleUglyFish'>]
玩家1 本次选择有 [<class 'chess_info.rabbit'>, <class 'chess_info.ant'>, <class 'chess_info.rabbit'>]


[chess_info.rabbit, chess_info.ant, chess_info.rabbit]

In [8]:
# player1.get_chess(rabbit())
# player1.get_chess(ant())
# player2.get_chess(ant())
# player2.get_chess(ant())

回合3


In [9]:
# player1.printStatus()
# player2.printStatus()
# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()
# player1.get_chess()
# player2.get_chess()

# player1.redraw()
# player2.upgrade_pop()
# player1.upgrade_chess()

回合4


回合5


回合6


回合7


In [10]:
    # [ [0,0] [0,1] [0,2] [0,3] [0,4] ]
    # [ [1,0] [1,1] [1,2] [1,3] [1,4] ]
    # [ [2,0] [2,1] [2,2] [2,3] [2,4] ]
    # [ [3,0] [3,1] [3,2] [3,3] [3,4] ]
    # [ [4,0] [4,1] [4,2] [4,3] [4,4] ]
    # [ [5,0] [5,1] [5,2] [5,3] [5,4] ]
# blueTeam = [bear(position = [3,2]), bee(position=[5,4]),sea_hedgehog(position=[3,4])]